# SISTEMA DE RECOMENDAÇÃO COM USO DE PROCESSAMENTO DE LINGUAGEM NATURAL

## Content-Based Recommendation System com uso de técnicas básicas de NLP

> Um sistema de recomendação baseado em conteúdo (Content-Based Recommendation System) é um algoritmo que usa o processamento de linguagem natural (NLP) para analisar o conteúdo de um item, como um artigo, curso ou filme, e encontrar outros itens semelhantes com base em seu conteúdo.<br>
<br>

> Por exemplo, se um usuário lê um artigo sobre tecnologia, um sistema de recomendação baseado em conteúdo com NLP pode analisar as palavras-chave e tópicos do artigo e encontrar outros artigos relacionados à tecnologia para recomendar ao usuário. Esse tipo de sistema é eficaz para recomendar itens com base nas preferências individuais do usuário e pode ajudar a aumentar a satisfação do usuário.<br>
<br>

> A principal vantagem aqui é a simplicidade da implementação, o que facilitaria um eventual deploy e reduziria a necessidade de se ter uma grande capacidade computacional para o processamento das recomendações em tempo real.<br>
<br>

> A desvantagem é que o sistema de recomendação proposto apresentaria as mesmas recomendações para usuários que fizessem os mesmos cursos, haja vista serem baseadas no conteúdo do último curso feito, não sendo consideradas as preferências gerais do usuário para personalização de recomendação.

### Fonte dos dados e dicionário de variáveis

> Conforme orientações dos professores, os dados foram obtidos de plataformas de dados abertos do governo, mais precisamente no site da Escola Virtual de Governo

> Logo, os dados utilizados aqui são abertos ao público e podem ser obtidos por meio do link: https://www.escolavirtual.gov.br/catalogo/exportar/csv.

> As variáveis utilizadas representam:

- id_curso: Identificador único do curso na plataforma;
- nome_curso: Título da capacitação;
- eixos_tematicos: Sub-divisão do eixo ao qual a capacitação pertence;
- competencias: Competencias profissionais abrangidas pela capacitação;
- certificador: Ente responsável pela plaicação do curso;
- conteudista: Ente repsonsável pela elaboração dos conteúdos do curso;
- carga_horaria: Medida em horas. Carga do curso;
- disponibilidade dias: Em quanto o tempo o curso pode ser feito; Medido em dias.
- tipo_oferta: Quem pode se inscrever na capacitação;
- apresentacao: Texto de apresentação do curso na plataforma;
- publico_alvo: Público-Alvo do curso;
- conteudo_programatico: Módulos e/ou capítulos que compõem a capacitação.

## Objetivos da Pesquisa:

> O objetivo é desenvolver um sistema de recomendação de cursos totalmente baseado em técnicas de NLP, empregando as máximo as técnicas ministradas durante o curso de NLP pelos professores Vinicius, Thiago e Luís.

> O sistema deverá apresentar recomendações no mínimo coerentes, as quais deverão ser avaliadas pelo operador, vez que nào existem rótulos e claramente trata-se de uma abordagem não-supervisionada.

> O sistema utilizará inicialmente técnicas simples e amplamente conhecidas de NLP visando testar a sua eficiência e criando um benchmark para comparações com novas versões que aplicam melhorias advindas de técnicas mais recentes e sofisticadas.

## Metodologia utilizada:

> O primeiro passo consistirá em identificar os dados que formarão o corpus;

> Em seguida, será aplicada uma etapa de pré-processamento dos dados, com vistas a tornar o texto o mais limpo possível para extração de caracteríticas, seguindo aquilo que foi ensinado pelos professors e adaptando as funções de processamento textual às particularidades dos dados e da tarefa;

> Como próximo passo, será feita a extração de características. Optarei por utilizar técnicas que domino com mais propriedade desde que foram ministradas, a saber o tfidf;

> Por conseguinte, aplicarei enquanto técnica de NLP a similaridade do cosseno por ter alguma experiência e entender o funcionamento para tarefas de semelhança de textos;

> A avaliação será feita diretamente, por meio da análise da similaridade e ajuste entre os 3 cursos recomendados e o curso-base que será dado ao modelo.

## Código e implementação:

> A partir de agora passarei à implementação com uso de python.

> Foram utilizadas a seguintes dependências no projeto:

    - python 3.10
    - scikit-learn==1.2.2
    - pandas==1.5.3
    - regex==2023.5.5
    - nltk==3.8.1

In [1]:
#Importando os dados das capacitacoes
import pandas as pd
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel


## Carregando dataset com os dadosda base da EVG.
df = pd.read_csv('/Users/pedroh.mello/Desktop/ENAP_CURSO_NLP/PROJETO_FINAL/catalogo_cursos_EVG.csv', sep='|')
df.head()

,id_curso,nome_curso,eixos_tematicos,competencias,certificador,conteudista,carga_horaria,disponibilidade_dias,tipo_oferta,apresentacao,publico_alvo,conteudo_programatico
0,448,Design Thinking Aplicado à Educação,Educação e Docência / Governo e Transformação ...,Inovação e mudança,Enap - Escola Nacional de Administração Pública,Enap - Escola Nacional de Administração Pública,25,30,Curso Aberto,Este curso é uma ótima oportunidade para você ...,"Servidores públicos que atuam na concepção, no...",Módulo 1: O contexto do Design Thinking na edu...
1,191,Supervisão de Estágio na Receita Federal do Br...,Gestão de Pessoas,NaN,Enap - Escola Nacional de Administração Pública,Receita Federal do Brasil,8,10,Curso Exclusivo,O curso Supervisão de Estágio foi produzido po...,Exclusivo para servidores da Receita Federal d...,Capítulo 1: IntroduçãoCapítulo 2: ContrataçãoC...
2,950,Promoção e Defesa dos Direitos das Pessoas LGB...,Direitos Humanos / Ética e Cidadania / Polític...,Foco nos resultados para os cidadãos / Geração...,Enap - Escola Nacional de Administração Pública,Ministério dos Direitos Humanos e Cidadania,30,30,Curso Aberto,"Para fortalecer a cultura de direitos humanos,...","Pessoas que atuam na garantia, defesa e promoç...",Módulo 1: Identidade de Gênero.Módulo 2: Orien...
3,535,Gerenciamento de serviços de TIC focado na Adm...,Governo e Transformação Digital / Inovação / G...,Foco nos resultados para os cidadãos / Inovaçã...,Enap - Escola Nacional de Administração Pública,Secretaria de Governo Digital,25,30,Curso Aberto,A tecnologia se encontra em todos os contextos...,"Agentes públicos, especialmente aqueles que sã...",Módulo 1: Desenvolvendo uma Visão Sobre Gerenc...
4,390,Etapas e Procedimentos do PNLD,Educação e Docência,NaN,Enap - Escola Nacional de Administração Pública,Ministério da Educação,30,30,Curso Aberto,O curso compõe a Trilha de Aprendizagem sobre ...,Todos os atores envolvidos nos processos de el...,Módulo 1: O livro como indutor da melhoria da ...


### Seleção das variáveis de interesse:

> Nem todas as variáveis serão utilizadas no estudo. O objetivo aqui é maximizar as informações textuais existentes sobre cada curso, de modo a ampliar o corpus e tentar captar ao máximo as relações textuais existentes.

> As variáveis numéricas não serão utilizadas, vez que poderiam atrapalhar no atingimento dos objetivos da tarefa de NLP em questão.

> Somente serão seleciondas do data set original as variáveis 'id_curso', 'nome_curso', 'apresentacao', 'conteudo_programatico', que serão guardadas em um novo dataframe chamado df_text.

In [2]:
##Selecionando apenas as colunas textuais descritivas, que são os dados de interesse
df_text = df[['id_curso', 'nome_curso', 'apresentacao', 'conteudo_programatico']].copy()

In [3]:
#Buscando por entradas nulas nos textos de apresentacao dos cursos
## Não existem entradas nulas.
df_text.isnull().sum()

id_curso                 0
nome_curso               0
apresentacao             0
conteudo_programatico    0
dtype: int64

#### Etapa de limpeza, transformação e compilação do texto

> A partir deste momento as seguintes atividades serão aplicadas ao texto com o uso de python:

- Padronização do texto para caracteres minúsculos;
- Eliminação das palavras Módulo e Capítulo e os respectivos números e ';' que as acompanham;
- Remoção de acentos e caracteres especiais;
- Remoção de espaços indevidos;
- Junção do nome do curso + conteúdo programático + texto de apresentação em uma única coluna de modo a maximizar as infos textuais e aumentar os documentos.

In [4]:
#Iniciando a limpeza do texto transformando tudo para minúsculo
df_text['nome_curso'] = df_text['nome_curso'].apply(lambda x: x.lower() if isinstance(x, str) else x)
df_text['apresentacao'] = df_text['apresentacao'].apply(lambda x: x.lower() if isinstance(x, str) else x)
df_text['conteudo_programatico'] = df_text['conteudo_programatico'].apply(lambda x: x.lower() if isinstance(x, str) else x)
df_text['conteudo_programatico'] = df_text['conteudo_programatico'].apply(lambda x: re.sub(r';', '; ', x))
df_text['conteudo_programatico'] = df_text['conteudo_programatico'].apply(lambda x: re.sub(r'\b(módulo|capítulo) \d+: ', '', x, flags=re.IGNORECASE))


In [5]:
#Função p/ limpeza de acentuação e caracteres especiais.

def traducao():
    string_origem = 'ãÃâÂáÁàÀêÊéÉèÈîÎíÍìÌïÏõÕôÔóÓòÒûÛúÚùÙüÜçÇñÑ/\'-"<>,.?!'
    string_destino = 'aaaaaaaaeeeeeeiiiiiiiioooooooouuuuuuuuccnn          '
    return string_origem.maketrans(string_origem, string_destino)

def clean_text(data):
    if isinstance(data, str):
        return data.translate(traducao())
    return data

#Aplicando as funcoes para limpeza dos textos
df_text['curso_transform'] = df_text['nome_curso'].apply(clean_text)
df_text['apresentacao_transform'] = df_text['apresentacao'].apply(clean_text)
df_text['conteudo_prog_transform'] = df_text['conteudo_programatico'].apply(clean_text)

In [6]:
# Ainda há necessidade de tirar as palavras modulo e capitulo das frases do conteudo prog. 
## Além disso, vou remover os numeros associados aos modulos e as pontuacoes totalmente;
def limpar_texto_compilado(texto):
    #remove pontuações restantes e espacpos extras
    texto_limpo = re.sub(r'[;,.]', '', texto)
    texto_limpo = re.sub(r'\s+', ' ', texto_limpo).strip()
    return texto_limpo

df_text['conteudo_prog_transform'] = df_text['conteudo_prog_transform'].apply(limpar_texto_compilado)


In [7]:
#Criando uma coluna que compila o nome do curso + texto de apresent. p/ maximizar infos textuais
def compilado(df_text):
    return df_text["curso_transform"]+" "+ df_text["apresentacao_transform"]+ " "+ df_text["conteudo_prog_transform"]

df_text['compilado_textual'] = compilado(df_text)

#### Assim ficou a configuração final dos dados após a aplicação do pré-prpcessamento:

In [8]:
df_text.head()

,id_curso,nome_curso,apresentacao,conteudo_programatico,curso_transform,apresentacao_transform,conteudo_prog_transform,compilado_textual
0,448,design thinking aplicado à educação,este curso é uma ótima oportunidade para você ...,o contexto do design thinking na educação.etap...,design thinking aplicado a educacao,este curso e uma otima oportunidade para voce ...,o contexto do design thinking na educacao etap...,design thinking aplicado a educacao este curso...
1,191,supervisão de estágio na receita federal do br...,o curso supervisão de estágio foi produzido po...,introduçãocapítulo 2: contrataçãocapítulo 3: a...,supervisao de estagio na receita federal do br...,o curso supervisao de estagio foi produzido po...,introducaocapitulo 2: contratacaocapitulo 3: a...,supervisao de estagio na receita federal do br...
2,950,promoção e defesa dos direitos das pessoas lgb...,"para fortalecer a cultura de direitos humanos,...",identidade de gênero.orientação afetiva-sexual...,promocao e defesa dos direitos das pessoas lgb...,para fortalecer a cultura de direitos humanos ...,identidade de genero orientacao afetiva sexual...,promocao e defesa dos direitos das pessoas lgb...
3,535,gerenciamento de serviços de tic focado na adm...,a tecnologia se encontra em todos os contextos...,desenvolvendo uma visão sobre gerenciamento de...,gerenciamento de servicos de tic focado na adm...,a tecnologia se encontra em todos os contextos...,desenvolvendo uma visao sobre gerenciamento de...,gerenciamento de servicos de tic focado na adm...
4,390,etapas e procedimentos do pnld,o curso compõe a trilha de aprendizagem sobre ...,o livro como indutor da melhoria da qualidade ...,etapas e procedimentos do pnld,o curso compoe a trilha de aprendizagem sobre ...,o livro como indutor da melhoria da qualidade ...,etapas e procedimentos do pnld o curso compoe ...


In [9]:
##verificandfo a saida apos o pre-process total. exempplo 1 documento.
df_text['compilado_textual'][4]

'etapas e procedimentos do pnld o curso compoe a trilha de aprendizagem sobre o pnld  possibilitando um acompanhamento mais efetivo das escolas e dos profissionais que atuam na educacao basica  promove a elaboracao dos planos pedagogicos  a escolha dos livros e as etapas de remanejamento  dentre outras praticas executadas na escola  o livro como indutor da melhoria da qualidade da educacao basica etapas procedimentos e funcionamento do pnld pnld na pratica recursos disponiveis de suporte ao pnld sistemas informatizados de apoio ao pnld'

### Extração de características e desenvolvimento da tarefa de NLP

> Neste momento, utilizarei apenas o TFIDF para extração das características dos textos dos cursos.

> O TFIDF pode ser utilizado por meio da biblioteca scikit-learn do python e foi amplamente ensinado pelos professores e utlizado durante as primeiras aulas do curso;

> Apesar de ser uma das técnicas mais rudimentares no campo do NLP, ainda apresenta bons resultads.

> A biblioteca nltk será responsável por listar automaticamente as stopwords em português que serão removidas para não atrapalharem a extração de características dos textos.

> Por conseguinte, o sistema fucnionará da seguinte maneira:
- À coluna compilado textual será aplicada a transformação em vetores TF-IDF. Esta etapa transformará os dados textuais em uma matriz tfidf onde as linhas representam os cursos e as colunas representam os termos, com os valores ponderados pelo tfidf.
- Em seguida, será feito o cálculo da similaridade do cosseno entre essas matrizes por meio da função built-in do scikit-learn 'linear kernel'.
- A similaridades do cosseno mede o ângulo enre dois vetores no espaço multidimensional. Quanto mais próximo de 1 for esse valor, mais alta será a similaridade.
- Após isto, a função get_recomend será responsável por comparar a similaridade do cosseno entre um curso inputado pelo usuário e a matriz completa, de maneira a retornar os 3 mais similares e os seus respectivos valores (excetuando-se o primeiro curso, que é sempre o mesmo do curso-base pois a similaridade será máxima).

In [10]:

#utilizando nltk para realizar acoes de NLP(remoc de stopwords)
stopwords_1 = nltk.corpus.stopwords.words('portuguese')
my_stopwords = ['apresentacao', 'curso', 'neste', 'capítulo', 'capitulo'
                ]
stopwords_1.extend(my_stopwords)

#vetorizando os textos e criando bag of words
tfidf_vectorizer = TfidfVectorizer(stop_words=stopwords_1)

In [11]:
#Calculando a similaridade do cosseno entre os vetores tfidf dos textos

tf_matrix = tfidf_vectorizer.fit_transform(df_text['compilado_textual'])
cosine_sim = linear_kernel(tf_matrix, tf_matrix)

In [25]:
tf_matrix.shape

(611, 6590)

In [22]:
cosine_sim

array([[1.        , 0.        , 0.00891448, ..., 0.00470345, 0.00908853,
        0.01561073],
       [0.        , 1.        , 0.        , ..., 0.0466949 , 0.        ,
        0.00695336],
       [0.00891448, 0.        , 1.        , ..., 0.00524394, 0.01715466,
        0.05904105],
       ...,
       [0.00470345, 0.0466949 , 0.00524394, ..., 1.        , 0.00284807,
        0.06810128],
       [0.00908853, 0.        , 0.01715466, ..., 0.00284807, 1.        ,
        0.00624859],
       [0.01561073, 0.00695336, 0.05904105, ..., 0.06810128, 0.00624859,
        1.        ]])

In [23]:
cosine_sim.shape

(611, 611)

In [12]:
## Chamando curso pelo id_curso
df1 = df_text.reset_index()
indice = pd.Series(df1.index, index=df1['id_curso'])

# construindo a funcao responsavel por recomendar curso 
def get_recomend(id_curso, cosine_sim=cosine_sim):
    idx = indice[id_curso]
    course_name = df1.loc[df1['id_curso'] == id_curso, 'nome_curso'].values[0].title()
    sim_score = list(enumerate(cosine_sim[idx]))
    sim_score = sorted(sim_score, key=lambda x: x[1], reverse=True)
    sim_score = sim_score[1:4]  # Pegando os 3 cursos mais similares, eliminando o primeiro que é ele mesmo
    sim_index = [i[0] for i in sim_score]
    
    recomend = pd.DataFrame({
        'Curso Recomendado': df1['nome_curso'].iloc[sim_index],
        'Sim_Coss': [score[1] for score in sim_score]
    }).reset_index(drop=True)
    
    print(f"As capacitações mais similares ao curso '{course_name}' são:\n")
    return recomend


#### A seguir um exemplo de como a função para recomendar cursos pode ser utilizada:

In [13]:
# Exemplo de uso da função
pd.set_option('display.max_colwidth', None)
print(get_recomend(486, cosine_sim))

As capacitações mais similares ao curso 'Nova Lei De Licitações: Visão Geral' são:

                                   Curso Recomendado  Sim_Coss
0  nova lei de licitações: planejamento e governança  0.427798
1          nova lei de licitações: gestão contratual  0.427230
2      nova lei de licitações: sanções ao fornecedor  0.415061


### Avaliando o fit das recomendações

> Por se tratar de um sistema de recomendação, a análise de quão bem o modelo performa deve ser feita por meio da avaliação da qualidade das recomendações extraídas do sistema a partir do seu fit aparente com o curso base.

> Desta feita, farei a extração de algumas recomendações para saber se guardam relação - ao menos indiretamente - com o curso base que foi passado à função.

##### **Exemplo de Avaliação 1: Curso-Base 'estatítica':**

> Conforme demonstrado a seguir, o modelo trouxe recomendações de similaridade muito ajustdas, vez que todos os cursos apresentados guardam nítida relação com o tema do curso-base

In [14]:
print(get_recomend(96, cosine_sim))

As capacitações mais similares ao curso 'Estatística' são:

                                            Curso Recomendado  Sim_Coss
0                                               macroeconomia  0.226345
1  estatística para análise de dados na administração pública  0.214377
2                                               microeconomia  0.199915


##### **Exemplo de Avaliação 2: Curso-Base 'Design Thinking Aplicado À Educação':**

> As recomendações de cursos parecem muito ajustdas, vez que todos os cursos apresentados guardam nítida relação com o tema do curso-base, assim como no exemplo anterior.

In [15]:
print(get_recomend(448, cosine_sim))

As capacitações mais similares ao curso 'Design Thinking Aplicado À Educação' são:

                                     Curso Recomendado  Sim_Coss
0               design thinking aplicado a bibliotecas  0.551885
1  princípios do design thinking e inovação em governo  0.462572
2   design sprint em projetos de transformação digital  0.274022


##### **Exemplo de Avaliação 3: Curso-Base 'Etapas E Procedimentos Do Pnld':**

> Conforme demonstrado a seguir, mais uma vez o modelo trouxe recomendações de similaridade muito ajustdas, vez que todos os cursos apresentados guardam nítida relação com o tema do curso-base.

> **Aqui ele demonstra uma de suas características mais importantes: A capacidade de recomendar cursos sequenciais;**

In [16]:
print(get_recomend(390, cosine_sim))

As capacitações mais similares ao curso 'Etapas E Procedimentos Do Pnld' são:

                                          Curso Recomendado  Sim_Coss
0                                     a importância do pnld  0.537163
1  pnld – programa nacional do livro e do material didático  0.449796
2                 a construção dos livros didáticos do pnld  0.402239


##### **Exemplo de Avaliação 4: Curso-Base 'Conceitos Essenciais Sobre Patologias Em Estruturas De Concreto':**

> Aqui o sistema demonstra um ponto fraco: Temas que possuem estrita relação ortográfica mas nenhuma relação semântica acabam sendo agrupados juntos pela máquina preditiva;

In [17]:
print(get_recomend(889, cosine_sim))

As capacitações mais similares ao curso 'Conceitos Essenciais Sobre Patologias Em Estruturas De Concreto' são:

                                               Curso Recomendado  Sim_Coss
0  estruturas organizacionais do poder executivo federal - siorg  0.105977
1                                    educação patrimonial e arte  0.086334
2                 fiscalização de projetos e obras de engenharia  0.085576


##### **Exemplo de Avaliação 5: Curso-Base 'S2Id - M3 - Usuário Federal - Solicitação De Recursos Para Obras De Reconstrução':**

> Mais uma vez ele demonstra uma excelente capacidade de lidar com cursos sequenciais.

In [18]:
print(get_recomend(782, cosine_sim))

As capacitações mais similares ao curso 'S2Id - M3 - Usuário Federal - Solicitação De Recursos Para Obras De Reconstrução' são:

                                                                    Curso Recomendado  \
0  s2id - m3 - usuário municipal - solicitação de recursos para obras de reconstrução   
1   s2id - m3 - usuário estadual - solicitação de recursos para obras de reconstrução   
2            s2id - m3 - usuário municipal - acompanhamento das obras de reconstrução   

   Sim_Coss  
0  0.341219  
1  0.331710  
2  0.312552  


## Conclusões:
> Após os passos anteriores, foi possível avaliar quais seriam as 3 recomendações de cursos que o sistema de NLP em questão seria capaz de prover.

> Apesar de utilizar técnicas iniciais de NLP, os resultados se mostram muito satisfatórios, haja vista a aparente baixa complexidade computacional do sistema e o baixo custo de processamento necessário para se obter as recomendações.

> Além disto, o sistema se mostrou muito bom na identificação e recomendação de cursos sequenciais.

> Apesar disso, o sistema apresenta limitações em relação à captação de contexto. Uma vez que o TFIDF não é a técnica mais adequada para captação de relações contextuais, temas que possuem relação ortográfica mas significados completamente diferentes acabam sendo agrupados juntos pela máquina preditiva.

> Isso fica evidente no exemplo de avaliação 4, onde o modelo apresentou alta similiaridade entre os cursos 'Conceitos Essenciais Sobre Patologias Em Estruturas De Concreto' e 'estruturas organizacionais do poder executivo federal - siorg', justamente por não captar a diferença entre estruturas organizacionais e de engenharia.

> Para tanto, uma importante oportunidade de melhoria a ser pesquisada seria aimplementação - na eta de extração de características - de modelos capazes de lidar com o estabelecimento de relações contextuais e 'memória' de relações de longo prazo, como uma rede neural bi-lstm ou até mesmo uma arquitetura de transformers.

> Uma próxima etapa interessante poderia ser o desenvolvimento de uma plaicação em streamlit que consumiria o modelo via API (endpoint construido para consumo do modelo) para testar o modelo à exaustão e verificar sua velocidade de resposta.

## ESPAÇO DESTINADO AOS TESTES DA FUNÇÃO:

> **Para obter alguma recomendação, escolha um dos cursos no catálogo anexo, copie seu id_curso e insira no argumento da função get_recomend abaixo:**

In [19]:
print(get_recomend(950, cosine_sim))

As capacitações mais similares ao curso 'Promoção E Defesa Dos Direitos Das Pessoas Lgbtqia+' são:

                                                         Curso Recomendado  \
0                               direitos humanos: uma declaração universal   
1                                             educação em direitos humanos   
2  direitos humanos e saúde mental - curso permanente damião ximenes lopes   

   Sim_Coss  
0  0.266931  
1  0.222446  
2  0.200156  
